In [146]:
from collections import defaultdict
import numpy as np
from time import time

In [4]:
def readData(filename):
        f = open(filename) ## May should be specific for different subtasks
        total_user_list = set()
        hash_data = defaultdict(int)
        for _, line in enumerate(f):
            arr = line.split("\t")
            hash_data[(int(arr[0]), int(arr[1]))] = 1
            total_user_list.add(int(arr[0]))
        total_user_list = list(total_user_list)
        hash_data = hash_data
        return total_user_list,hash_data

In [9]:
def arrangePositiveData(hash_data):
    """
    用户交互的item
    """
    positive_data = defaultdict(set)
    total_data = set()
    for (u, i) in hash_data:
        total_data.add((u, i))
        positive_data[u].add(i)
    return positive_data,len(total_data)

In [14]:
def generateTrainNegative(num_items,num_negatives,hash_data):
    negative_data = defaultdict(set)
    total_data = set()
    for (u, i) in hash_data:
        total_data.add((u, i))
        for _ in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in hash_data:
                j = np.random.randint(num_items)
            negative_data[u].add(j)
            total_data.add((u, j))
    terminal_flag = 1
    return negative_data,terminal_flag

In [19]:
num_items = 38342
num_negatives = 8
negative_data,terminal_flag = generateTrainNegative(num_items,num_negatives,hash_data)

In [24]:
def getVTRankingOneBatch(positive_data,negative_data,total_user_list):
    """
    user_list:userid*len(pos[userid])+userid*len(neg[userid])
    item_list:neg[userid]+neg[userid]
    """
    user_list = []
    item_list = []
    labels_list = []
    for u in total_user_list:
        user_list.extend([u] * len(positive_data[u]))
        item_list.extend(positive_data[u])
        labels_list.extend([1] * len(positive_data[u]))
        user_list.extend([u] * len(negative_data[u]))
        item_list.extend(negative_data[u])
        labels_list.extend([0] * len(negative_data[u]))
        
    user_list = np.reshape(user_list, [-1, 1])
    item_list = np.reshape(item_list, [-1, 1])
    labels_list = np.reshape(labels_list, [-1, 1])
    return user_list,item_list,labels_list

In [26]:
user_list,item_list,labels_list=getVTRankingOneBatch(positive_data,negative_data,total_user_list)

In [41]:
def readSocialNeighbors(links_filename, friends_flag=1):
    social_neighbors = defaultdict(set)
    links_file = open(links_filename)
    for _, line in enumerate(links_file):
        tmp = line.split('\t')
        u1, u2 = int(tmp[0]), int(tmp[1])
        social_neighbors[u1].add(u2)
        if friends_flag == 1:
            social_neighbors[u2].add(u1)
    return social_neighbors

In [42]:
links_filename = './data/yelp.links'
social_neighbors=readSocialNeighbors(links_filename)

In [44]:
def generateSocialNeighborsSparseMatrix(social_neighbors):
    """
     social_neighbors_indices_list:user和每个trust friend
     social_neighbors_values_list：user和每个friend的 系数   1/len(friend)
    """
    social_neighbors_indices_list = []
    social_neighbors_values_list = []
    social_neighbors_dict = defaultdict(list)
    for u in social_neighbors:
        social_neighbors_dict[u] = sorted(social_neighbors[u])
            
    user_list = sorted(list(social_neighbors.keys()))
    for user in user_list:
        for friend in social_neighbors_dict[user]:
            social_neighbors_indices_list.append([user, friend])
            social_neighbors_values_list.append(1.0/len(social_neighbors_dict[user]))
    social_neighbors_indices_list = np.array(social_neighbors_indices_list).astype(np.int64)
    social_neighbors_values_list = np.array(social_neighbors_values_list).astype(np.float32)
    return social_neighbors_indices_list,social_neighbors_values_list

In [45]:
social_neighbors_indices_list,social_neighbors_values_list = generateSocialNeighborsSparseMatrix(social_neighbors)

In [49]:
def generateConsumedItemsSparseMatrix(positive_data):
    '''
        Generate Consumed Items Sparse Matrix Indices and Values
    '''
    consumed_items_indices_list = []
    consumed_items_values_list = []
    consumed_items_dict = defaultdict(list)
    for u in positive_data:
        consumed_items_dict[u] = sorted(positive_data[u])
    user_list = sorted(list(positive_data.keys()))
    for u in user_list:
        for i in consumed_items_dict[u]:
            consumed_items_indices_list.append([u, i])
            consumed_items_values_list.append(1.0/len(consumed_items_dict[u]))
    consumed_items_indices_list = np.array(consumed_items_indices_list).astype(np.int64)
    consumed_items_values_list = np.array(consumed_items_values_list).astype(np.float32)
    return consumed_items_indices_list,consumed_items_values_list

In [50]:
consumed_items_indices_list,consumed_items_values_list = generateConsumedItemsSparseMatrix(positive_data)

In [119]:
total_user_list,hash_data = readData('./data/yelp.train.rating')
positive_data,total_data = arrangePositiveData(hash_data)
negative_data,terminal_flag = generateTrainNegative(num_items,num_negatives,hash_data)

In [144]:
num_users=17237
num_items=38342
dimension = 32
learning_rate=0.001
epochs = 10

In [143]:
model = DiffNet(num_users,num_items,dimension,learning_rate,
               social_neighbors_indices_list,social_neighbors_values_list,
               consumed_items_indices_list,consumed_items_values_list)

In [ ]:
with tf.Session() as sess:
    sess.run(model.init)
    
    # train
    for epoch in range(1, epochs+1):
        tmp_train_loss = []
        t0 = time()
        
        

In [147]:
def get_Batch(data, label, batch_size):
    print(data.shape, label.shape)
    input_queue = tf.train.slice_input_producer([data, label], num_epochs=1, shuffle=True, capacity=32 ) 
    x_batch, y_batch = tf.train.batch(input_queue, batch_size=batch_size, num_threads=1, capacity=32, allow_smaller_final_batch=False)
    return x_batch, y_batch

In [148]:
x_batch, y_batch = get_Batch(positive_data,labels_list,64)

AttributeError: 'collections.defaultdict' object has no attribute 'shape'

In [149]:
positive_data

defaultdict(set,
            {0: {4000, 5490, 7000, 21323, 28202},
             1: {5941, 10953, 16243, 17248, 30103, 35309},
             2: {2534,
              2607,
              3870,
              3896,
              4865,
              6259,
              9224,
              15641,
              16184,
              21145,
              21297,
              25543,
              25800,
              34431},
             3: {1084, 14856, 15455, 17429, 31975, 32587},
             4: {1800,
              2279,
              2507,
              3164,
              3782,
              4187,
              4241,
              4579,
              4826,
              5078,
              5959,
              5984,
              6101,
              6994,
              7447,
              7714,
              7992,
              8604,
              9663,
              10404,
              10822,
              11250,
              11948,
              12153,
              12785,
              1

In [56]:
import tensorflow as tf
import numpy as np

In [142]:
class DiffNet():
    def __init__(self,num_users,num_items,dimension,learning_rate,
                social_neighbors_indices_input,social_neighbors_values_input,
                consumed_items_indices_input,consumed_items_values_input,
                ):
        self.num_users = num_users
        self.num_items = num_items
        self.dimension = dimension  #用户嵌入向量维度
        self.learning_rate = learning_rate
        
        self.social_neighbors_indices_input=social_neighbors_indices_input
        self.social_neighbors_values_input=social_neighbors_values_input
        self.consumed_items_indices_input=consumed_items_indices_input
        self.consumed_items_values_input = consumed_items_values_input
        
        #用户与用户的二部图
        self.social_neighbors_dense_shape = np.array([self.num_users, self.num_users]).astype(np.int64)
        #用户与项的二部图
        self.consumed_items_dense_shape = np.array([self.num_users, self.num_items]).astype(np.int64)
        
        self.social_neighbors_sparse_matrix= tf.SparseTensor(
            indices= self.social_neighbors_indices_input,
            values = self.social_neighbors_values_input,
            dense_shape = self.social_neighbors_dense_shape
        )
        
        self.consumed_items_sparse_matrix = tf.SparseTensor(
            indices = self.consumed_items_indices_input,
            values = self.consumed_items_values_input,
            dense_shape=self.consumed_items_dense_shape
        )
        
        
        self.user_review_vector_matrix = np.load('./data/user_vector.npy')
        self.item_review_vector_matrix=np.load('./data/item_vector.npy')
        
        self.initializeNodes()
        self.constructTrainGraph()
        self.saveVariables()
        self.defineMap()
        
        
    
    def initializeNodes(self):
        self.item_input = tf.placeholder("int32", [None, 1]) # Get item embedding from the core_item_input
        self.user_input = tf.placeholder("int32", [None, 1]) # Get user embedding from the core_user_input
        self.labels_input = tf.placeholder("float32", [None, 1])
        # user初始嵌入矩阵
        self.user_embedding = tf.Variable(
            tf.random_normal([self.num_users, self.dimension], stddev=0.01),
            name='user_embedding')
        
        # item初始嵌入矩阵
        self.item_embedding = tf.Variable(
            tf.random_normal([self.num_items, self.dimension], stddev=0.01),
            name='item_embedding')
        
        self.user_review_vector_matrix = tf.constant(self.user_review_vector_matrix, dtype=tf.float32)
        self.item_review_vector_matrix = tf.constant(self.item_review_vector_matrix, dtype=tf.float32)
        
        self.reduce_dimension_layer = tf.layers.Dense(self.dimension, activation=tf.nn.sigmoid, name='reduce_dimension_layer')

        self.item_fusion_layer = tf.layers.Dense(self.dimension, activation=tf.nn.sigmoid, name='item_fusion_layer')
        self.user_fusion_layer = tf.layers.Dense(self.dimension, activation=tf.nn.sigmoid, name='user_fusion_layer')
        
    def constructTrainGraph(self):
        ## 将user和item的feature matrix介入到原始matrix
        first_user_review_vector_matrix = self.convertDistribution(self.user_review_vector_matrix)
        first_item_review_vector_matrix = self.convertDistribution(self.item_review_vector_matrix)
        
        self.user_reduce_dim_vector_matrix = self.reduce_dimension_layer(first_user_review_vector_matrix)
        self.item_reduce_dim_vector_matrix = self.reduce_dimension_layer(first_item_review_vector_matrix)
        
        second_user_review_vector_matrix = self.convertDistribution(self.user_reduce_dim_vector_matrix)
        second_item_review_vector_matrix = self.convertDistribution(self.item_reduce_dim_vector_matrix)
        
        self.final_item_embedding = self.fusion_item_embedding = self.item_embedding + second_item_review_vector_matrix
        user_embedding_from_consumed_items = self.generateUserEmebddingFromConsumedItems(self.final_item_embedding)
        
        
        
        self.fusion_user_embedding = self.user_embedding + second_user_review_vector_matrix
        first_gcn_user_embedding = self.generateUserEmbeddingFromSocialNeighbors(self.fusion_user_embedding)
        second_gcn_user_embedding = self.generateUserEmbeddingFromSocialNeighbors(first_gcn_user_embedding)
        
        self.final_user_embedding = second_gcn_user_embedding + user_embedding_from_consumed_items
        
        latest_user_latent = tf.gather_nd(self.final_user_embedding, self.user_input)
        latest_item_latent = tf.gather_nd(self.final_item_embedding, self.item_input)
        
        predict_vector = tf.multiply(latest_user_latent, latest_item_latent)
        self.prediction = tf.sigmoid(tf.reduce_sum(predict_vector, 1))

        self.loss = tf.nn.l2_loss(self.labels_input - self.prediction)
        self.opt_loss = tf.nn.l2_loss(self.labels_input - self.prediction)
        self.opt = tf.train.AdamOptimizer(self.learning_rate).minimize(self.opt_loss)
        self.init = tf.global_variables_initializer()
        
        
    #进行BN
    def convertDistribution(self, x):
        mean, var = tf.nn.moments(x, axes=[0, 1])
        y = (x - mean) * 0.2 / tf.sqrt(var)
        return y
    
    def generateUserEmebddingFromConsumedItems(self, current_item_embedding):
        user_embedding_from_consumed_items = tf.sparse_tensor_dense_matmul(
            self.consumed_items_sparse_matrix, current_item_embedding
        )
        return user_embedding_from_consumed_items
    
    def generateUserEmbeddingFromSocialNeighbors(self, current_user_embedding):
        user_embedding_from_social_neighbors = tf.sparse_tensor_dense_matmul(
            self.social_neighbors_sparse_matrix, current_user_embedding
        )
        return user_embedding_from_social_neighbors
    
    
    def saveVariables(self):
        ############################# Save Variables #################################
        variables_dict = {}
        variables_dict[self.user_embedding.op.name] = self.user_embedding
        variables_dict[self.item_embedding.op.name] = self.item_embedding

        for v in self.reduce_dimension_layer.variables:
            variables_dict[v.op.name] = v
                
        self.saver = tf.train.Saver(variables_dict)
        
    
    def defineMap(self):
        map_dict = {}
        map_dict['train'] = {
            self.user_input: 'USER_LIST', 
            self.item_input: 'ITEM_LIST', 
            self.labels_input: 'LABEL_LIST'
        }
        
        map_dict['val'] = {
            self.user_input: 'USER_LIST', 
            self.item_input: 'ITEM_LIST', 
            self.labels_input: 'LABEL_LIST'
        }

        map_dict['test'] = {
            self.user_input: 'USER_LIST', 
            self.item_input: 'ITEM_LIST', 
            self.labels_input: 'LABEL_LIST'
        }

        map_dict['eva'] = {
            self.user_input: 'EVA_USER_LIST', 
            self.item_input: 'EVA_ITEM_LIST'
        }

        map_dict['out'] = {
            'train': self.loss,
            'val': self.loss,
            'test': self.loss,
            'eva': self.prediction
        }

        self.map_dict = map_dict

In [91]:
test = np.load('./data/user_vector.npy')

In [117]:

x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=16)
y = linear_model(x)
 
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[-1.048022   -1.8628637  -2.3610103  -0.01957721  0.17956007 -0.6580874
  -1.5625     -0.8215864   0.48028958  1.2311993  -2.2170815  -1.7174402
  -1.8151314  -0.5628109  -0.51908255 -0.3295083 ]
 [-1.9761099  -4.0382447  -4.6557693  -0.5093671   0.88320816 -1.9862868
  -3.983462   -2.2856822   0.6303438   2.0222645  -4.452523   -4.22628
  -3.541888   -1.3307154  -0.86340284 -1.966671  ]]


<tf.Tensor 'Const:0' shape=(10, 10) dtype=float32>